# Packages

In [ ]:
using FinEtools, FinEtoolsAcoustics, LinearAlgebra, StatsBase, HDF5

# Initial setup

In [ ]:
rho = 1.21 * phun("kg/m^3") # mass density
c  = 343.0 * phun("m/s") # sound speed
bulk =  c ^ 2 * rho
frequency = 500.0 # frequency of the incident wave, Hz
omega = 2 * pi * frequency
dt = 1.0 / frequency / 10
tfinal = 115 * dt
println("Step takes $(sciNotation(dt, 3)) s")
println("Final time: $(sciNotation(reachTime, 3)) s")
nsteps = round(tfinal / dt) + 1;

# Mesh

In [ ]:
# list of meshes
Meshes = Array{Tuple{FENodeSet, AbstractFESet}, 1}()
# Q4 discretization of large rectangular domain divisions
push!(Meshes, Q4quadrilateral([0.0 0.0; 1.2 2.6], 12, 26)) # 1
push!(Meshes, Q4quadrilateral([0.0 2.6; 1.6 5.45], 15, 30)) # 2
push!(Meshes, Q4quadrilateral([1.35 0.0; 4.05 2.6], 30, 24)) # 3
push!(Meshes, Q4quadrilateral([1.6 2.6; 4.05 5.8], 25, 26)) # 4
push!(Meshes, Q4quadrilateral([4.25 1.2; 5.25 4.5], 10, 35)) # 5
push!(Meshes, Q4quadrilateral([4.05 4.5; 5.25 5.3], 12, 8)) # 6
push!(Meshes, Q4quadrilateral([4.25 5.3; 5.25 5.85], 10, 4)) # 7
push!(Meshes, Q4quadrilateral([5.25 1.2; 7.0 3.92], 22, 25)) # 8
# merge meshes
fens, outputfes = mergenmeshes(Meshes, 0.05)
# concatenate connectivities
fes = cat(outputfes[5], cat(outputfes[6], cat(outputfes[7], outputfes[8])))
fes = cat(outputfes[1], cat(outputfes[2], cat(outputfes[3], cat(outputfes[4], fes))))
println("$(size(fens.xyz, 1)) nodes")
println("$(size(fes.conn, 1)) elements")
println("$(size(fens.xyz, 1) * round(Int, tfinal / dt)) dataset points (using Q4)")

# Subdomain selection

In [ ]:
# Identify boundary finite element set
bfes = meshboundary(fes)
# In case there are any unconnected nodes, remove them, and renumber the elements.
connected = findunconnnodes(fens, fes)
fens, new_numbering = compactnodes(fens, connected)
fess = renumberconn!(fes, new_numbering)
# The geometry and the solution (pressure) fields
geom = NodalField(fens.xyz)
P = NodalField(zeros(FCplxFlt, size(fens.xyz, 1), 1))

# Visualize geometry

In [ ]:
# Export three VTK files: one for the interior of the fluid, and one for the boundary
vtkexportmesh("interior.vtk", fes.conn, fens.xyz, FinEtools.MeshExportModule.VTK.Q4)

# Setup discrete model

In [ ]:
# Number the degrees of freedom in the pressure field.
numberdofs!(P)
# Create the finite element machine for the fluid.
material = MatAcoustFluid(bulk, rho)
femm  =  FEMMAcoust(IntegDomain(fes, GaussRule(3, 2)), material)
# Use the machine calculate the acoustic stiffness and mass matrices.
S  =  acousticstiffness(femm, geom, P)
C  =  acousticmass(femm, geom, P)
# "damping" matrix for the absorbing boundary conditions (ABCs)
D  =  acousticABC(
  FEMMAcoustSurf(IntegDomain(bfes, GaussRule(2, 2)), material), geom, P
)
loadNode = subset(fes, selectelem(fens, fes, withnodes = [2049, 2050, 2075, 2074]))
dipfemm = FEMMAcoustSurf(IntegDomain(loadNode, GaussRule(2, 2)), material)
# harmonic point load
function pointLoad(dpdn, xyz, J, label, t)::Float64
  dpdn[1] = -rho * 0.02 * sin(omega * t)
end

# Time stepping

In [ ]:
# Solve the transient acoustics equations.
# The loop executes inside this local scope
t = 0.0 # Initial time
# initialize variable for dataset.
# dataset is (N x 4) matrix with rows: [nodeX nodeY time nodalPressure]ₙ
# N is the amount of samples: number of nodes * number of time steps
data = Float32[0.0 0.0 0.0 0.0]
pressureMat = zeros(Float32, (nnodes(P), 1))
P1 = let
  P0 = deepcopy(P)
  P0.values .= 0.0 # initially all pressure is zero
  vP0 = gathersysvec(P0)
  vQ0 = zeros(eltype(vP0), size(vP0))
  # The `P1` field will be the output of this computation:
  # the final value of the pressure field
  P1 = deepcopy(P0)
  fi = ForceIntensity( # Initial load
    FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
  )
  La0 = distribloads(dipfemm, geom, P1, fi, 2)
  A = (2.0 / dt) * S + D + (dt / 2.0) * C
  step = 0
  while t <= tfinal
    step += 1
    global t += dt
    # Update load
    fi = ForceIntensity( # Initial load
      FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
    )
    La1 = distribloads(dipfemm, geom, P1, fi, 2)
    # Solve for the rate of the pressure
    vQ1 = A \ ((2 / dt) * (S * vQ0) - D * vQ0 - C * (2 * vP0 + (dt / 2) * vQ0) + La0 + La1)
    
    # Update the value of the pressure
    vP1 = vP0 + (dt / 2) * (vQ0 + vQ1)
    
    # Swap variables for next step  
    vP0 = deepcopy(vP1)
    vQ0 = deepcopy(vQ1)
    P1 = scattersysvec!(P1, vec(vP1))
    P0 = deepcopy(P1)
    La0 = deepcopy(La1)
    # concatenate current pressure field to dataset.
    global data = vcat(
      data, hcat(fens.xyz, fill(t, size(fens.xyz, 1)), real.(P1.values) |> vec)
    )
    global pressureMat = hcat(pressureMat, real.(P1.values))
  end
  P1 # Return the final pressure
end
# discard matrix initializations
data = data[2:end, :]
pressureMat = pressureMat[:, 2:end];

# Export pressure fields

In [ ]:
# vtkexportmesh(File, fes.conn, geom.values, FinEtools.MeshExportModule.VTK.Q4; scalars = [( "realP", pressureMat),])
for fileID in axes(pressureMat, 2)
  vtkexportmesh(
    "./vtkFiles/wavePropagation$fileID.vtk",
    fes.conn, geom.values, FinEtools.MeshExportModule.VTK.Q4;
    scalars = [( "pressure", pressureMat[:, fileID] |> vec)]
  )
end
# @async run(`"paraview.exe" $(readdir(pwd(); join = true)[end])`)

# Data statistics

In [ ]:
for i in 1:4
  @show data[:, i] |> unique |> length
  statsum(data[:, i])
  println()
end

In [ ]:
# dataStandard = standardize(ZScoreTransform, data, dims = 1)
dataStandard = standardize(UnitRangeTransform, data, dims = 1)

In [ ]:
mapslices(statsum, dataStandard; dims = [1]);

# Save dataset

In [ ]:
# create file
dataFile = h5open(datasetPath * "./dataset", "w")
# without normalization
create_dataset(quickTOdata, "data", zeros(Float32, size(data)))
create_dataset(quickTOdata, "normalizedData", zeros(Float32, size(data)))